<a href="https://colab.research.google.com/github/Sandutta2020/Pyspark/blob/Master/pyspark_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=620e43a5b67a1e3deb7cb983ed6f4dd362ca4b08b9a2c3ea9e65173298f291e8
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

In [ ]:
!git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 13.02 KiB | 4.34 MiB/s, done.


In [ ]:
!ls diabetes_dataset


diabetes.csv  new_test.csv


In [ ]:
df =spark.read.csv("/content/diabetes_dataset/diabetes.csv", header=True, inferSchema=True)

In [ ]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [ ]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
print((df.count(), len(df.columns)))

(2000, 9)


In [ ]:
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [ ]:
df.groupby("Outcome").count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [ ]:
for col in df.columns:
  print(col+":", df[df[col].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [ ]:
def count_zeros():
  columns_list = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
  for i in columns_list:
    print(i+":", df[df[i]==0].count())


In [ ]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [ ]:
for i in df.columns[1:6]:
    data =df.agg({i:"mean"}).first()[0]
    print("mean value for {} is {}".format(i, int(data)))

mean value for Glucose is 121
mean value for BloodPressure is 69
mean value for SkinThickness is 20
mean value for Insulin is 80
mean value for BMI is 32


In [ ]:
from pyspark.sql.functions import *
for i in df.columns[1:6]:
    data =df.agg({i:"mean"}).first()[0]
    print("mean value for {} is {}".format(i, int(data)))
    df = df.withColumn(i, when(col(i)==0, int(data)).otherwise(col(i)))

mean value for Glucose is 121
mean value for BloodPressure is 69
mean value for SkinThickness is 20
mean value for Insulin is 80
mean value for BMI is 32


In [ ]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


In [ ]:
for col in df.columns:
  print("Correlation to outcome for", col, df.stat.corr("Outcome", col))

Correlation to outcome for Pregnancies 0.22443699263363961
Correlation to outcome for Glucose 0.48796646527321064
Correlation to outcome for BloodPressure 0.17171333286446713
Correlation to outcome for SkinThickness 0.1659010662889893
Correlation to outcome for Insulin 0.1711763270226193
Correlation to outcome for BMI 0.2827927569760082
Correlation to outcome for DiabetesPedigreeFunction 0.1554590791569403
Correlation to outcome for Age 0.23650924717620253
Correlation to outcome for Outcome 1.0


In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"], outputCol="features")

In [ ]:
output_data=assembler.transform(df)

In [ ]:
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

In [ ]:
from pyspark.ml.classification import LogisticRegression
final_data = output_data.select("features", "Outcome")

In [ ]:
train, test = final_data.randomSplit([0.7, 0.3])
models = LogisticRegression(labelCol="Outcome")
model = models.fit(train)

In [ ]:
summary =model.summary

In [ ]:
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|            Outcome|         prediction|
+-------+-------------------+-------------------+
|  count|               1357|               1357|
|   mean| 0.3426676492262343|0.26823876197494473|
| stddev|0.47477641307394114|0.44320591464204034|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.evaluate(test)

In [ ]:
predictions.predictions.show(20)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,20...|      0|[3.78667284509708...|[0.97783167000461...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.40180103202139...|[0.91696453813937...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.58886474322513...|[0.93014148607824...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.58886474322513...|[0.93014148607824...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.75265118453832...|[0.94006290426091...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.75265118453832...|[0.94006290426091...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.71202929644431...|[0.93773274463613...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.71202929644431...|[0.93773274463613...|       0.0|
|[0.0,93.0,100.0,3...|      0|[1.31864645228459...|[0.78895642330879...|    

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Outcome")
evaluator.evaluate(model.transform(test))

0.8289717411906609

In [ ]:
model.save("model")

In [ ]:
!ls

diabetes_dataset  model  sample_data


In [ ]:
import pyspark.ml.classification as classification
model = classification.LogisticRegressionModel.load("model")